In [3]:
import os
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient

model_client = OpenAIChatCompletionClient(
    api_key=os.environ['OPENAI_API_KEY'],
    base_url=os.environ['OPENAI_API_BASE'],
    model='gpt-4o-2024-11-20',
)

In [4]:
agent = AssistantAgent("assistant", model_client)
print(await agent.run(task="Say 'Hello World!'"))

TaskResult(messages=[TextMessage(source='user', models_usage=None, content="Say 'Hello World!'", type='TextMessage'), TextMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=41, completion_tokens=3), content='Hello World!', type='TextMessage')], stop_reason=None)


/Users/jizhang/Library/Caches/pypoetry/virtualenvs/ai-sandbox-aoUc62G5-py3.12/lib/python3.12/site-packages/autogen_agentchat/agents/_assistant_agent.py:416: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o. Model mapping in autogen_ext.models.openai may be incorrect.
  model_result = await self._model_client.create(


In [ ]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.agents.web_surfer import MultimodalWebSurfer

assistant = AssistantAgent("assistant", model_client)
web_surfer = MultimodalWebSurfer("web_surfer", model_client)
user_proxy = UserProxyAgent("user_proxy")
termination = TextMentionTermination("exit") # Type 'exit' to end the conversation.
team = RoundRobinGroupChat([web_surfer, assistant, user_proxy], termination_condition=termination)
await Console(team.run_stream(task="Find information about AutoGen and write a short summary."))


In [4]:
async def get_weather(city: str) -> str:
    """Get the weather for a given city."""
    return f"The weather in {city} is 73 degrees and Sunny."

agent = AssistantAgent(
    name="weather_agent",
    model_client=model_client,
    tools=[get_weather],
    system_message="You are a helpful assistant.",
    reflect_on_tool_use=True,
    model_client_stream=True,  # Enable streaming tokens from the model client.
)

await Console(agent.run_stream(task="What is the weather in New York?"))

---------- user ----------
What is the weather in New York?
---------- weather_agent ----------
[FunctionCall(id='call_X1R20HEr2cBui3SLnmh1TWbz', arguments='{"city":"New York"}', name='get_weather')]
---------- weather_agent ----------
[FunctionExecutionResult(content='The weather in New York is 73 degrees and Sunny.', call_id='call_X1R20HEr2cBui3SLnmh1TWbz', is_error=False)]
---------- weather_agent ----------
The current weather in New York is 73 degrees and sunny.


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='What is the weather in New York?', type='TextMessage'), ToolCallRequestEvent(source='weather_agent', models_usage=RequestUsage(prompt_tokens=66, completion_tokens=0), content=[FunctionCall(id='call_X1R20HEr2cBui3SLnmh1TWbz', arguments='{"city":"New York"}', name='get_weather')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='weather_agent', models_usage=None, content=[FunctionExecutionResult(content='The weather in New York is 73 degrees and Sunny.', call_id='call_X1R20HEr2cBui3SLnmh1TWbz', is_error=False)], type='ToolCallExecutionEvent'), TextMessage(source='weather_agent', models_usage=RequestUsage(prompt_tokens=64, completion_tokens=14), content='The current weather in New York is 73 degrees and sunny.', type='TextMessage')], stop_reason=None)

In [4]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console

primary_agent = AssistantAgent(
    "primary",
    model_client=model_client,
    system_message="You are a helpful AI assistant.",
)

critic_agent = AssistantAgent(
    "critic",
    model_client=model_client,
    system_message="Provide constructive feedback. Respond with 'APPROVE' when your feedbacks are addressed.",
)

text_termination = TextMentionTermination("APPROVE")
team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=text_termination)
await Console(team.run_stream(task="Write a short poem about the fall season.")) 

---------- user ----------
Write a short poem about the fall season.


---------- primary ----------
Golden leaves drift through the air,  
Whispers of autumn everywhere.  
Crisp winds dance, the world turns slow,  
Beneath a blush of amber glow.  

Pumpkin spice and fires bright,  
Cool, long evenings, stars alight.  
Nature sighs, a fleeting call,  
In quiet grace, we welcome fall.  


/Users/jizhang/Library/Caches/pypoetry/virtualenvs/ai-sandbox-aoUc62G5-py3.12/lib/python3.12/site-packages/autogen_agentchat/agents/_assistant_agent.py:416: UserWarning: Resolved model mismatch: gpt-4o-2024-11-20 != openai/gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect.
  model_result = await self._model_client.create(


---------- critic ----------
This is a lovely poem with vivid imagery and a serene tone that captures the essence of autumn. The flow and word choices evoke a warm and familiar feeling. However, you might consider varying the rhythm slightly to avoid any potential monotony, as the consistent rhyme scheme—while beautiful—could limit dynamic contrast. Additionally, exploring even more unique or unexpected imagery could elevate the poem further and provide a fresh take on the common fall theme.  

Once any adjustments or refinements are made, let me know, and we can revisit.
---------- primary ----------
Thank you for the thoughtful critique! I appreciate your insights and would be happy to refine the poem by incorporating more varied rhythm and fresher imagery while keeping the heart of the piece intact. Here's a revised version:

---

Shadows stretch long on amber streets,  
The earth hums low beneath our feet.  
Leaves tumble like whispers, soft and free,  
A fleeting dance with myster

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=28, completion_tokens=74), content='Golden leaves drift through the air,  \nWhispers of autumn everywhere.  \nCrisp winds dance, the world turns slow,  \nBeneath a blush of amber glow.  \n\nPumpkin spice and fires bright,  \nCool, long evenings, stars alight.  \nNature sighs, a fleeting call,  \nIn quiet grace, we welcome fall.  ', type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=118, completion_tokens=106), content='This is a lovely poem with vivid imagery and a serene tone that captures the essence of autumn. The flow and word choices evoke a warm and familiar feeling. However, you might consider varying the rhythm slightly to avoid any potential monotony, as the consistent rhyme scheme—while beautiful—could limit dynamic contrast. Addi